In [2]:
import pandas as pd

In [16]:
df = pd.read_csv(
    "individual+household+electric+power+consumption/household_power_consumption.txt",      
    sep=';',
    parse_dates={'datetime': ['Date', 'Time']},
    infer_datetime_format=True,
    na_values='?',
    low_memory=False
)

C:\Users\chaud\AppData\Local\Temp\ipykernel_18372\3558349531.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
C:\Users\chaud\AppData\Local\Temp\ipykernel_18372\3558349531.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(
C:\Users\chaud\AppData\Local\Temp\ipykernel_18372\3558349531.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(


In [17]:
df.head()

,datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [18]:
df.isnull().sum()

datetime                     0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [19]:
df_clean = df.dropna().copy()

In [20]:
num_cols = df_clean.columns.drop('datetime')
df_clean.loc[:, num_cols] = df_clean.loc[:, num_cols].astype(float)
df_clean.loc[:, 'kwh'] = df_clean['Global_active_power'] / 60
df_clean.set_index('datetime', inplace=True)

In [21]:
daily_features = df_clean.resample('D').agg({
    'kwh': 'sum',
    'Voltage': 'mean',
    'Global_intensity': 'mean',
    'Sub_metering_1': 'sum',
    'Sub_metering_2': 'sum',
    'Sub_metering_3': 'sum'
})

daily_features.columns = [
    'daily_kwh',
    'avg_voltage',
    'avg_intensity',
    'kitchen_kwh',
    'laundry_kwh',
    'other_kwh'
]

In [22]:
daily_features = daily_features.reset_index()

daily_features['day_of_week'] = daily_features['datetime'].dt.dayofweek
daily_features['month'] = daily_features['datetime'].dt.month
daily_features['is_weekend'] = daily_features['day_of_week'].isin([5, 6]).astype(int)

In [23]:
daily_features.head()

,datetime,daily_kwh,avg_voltage,avg_intensity,kitchen_kwh,laundry_kwh,other_kwh,day_of_week,month,is_weekend
0,2006-12-16,20.152933,236.243763,13.082828,0.0,546.0,4926.0,5,12,1
1,2006-12-17,56.507667,240.087028,9.999028,2033.0,4187.0,13341.0,6,12,1
2,2006-12-18,36.730433,241.231694,6.421667,1063.0,2621.0,14018.0,0,12,0
3,2006-12-19,27.769900,241.999313,4.926389,839.0,7602.0,6197.0,1,12,0
4,2006-12-20,37.095800,242.308062,6.467361,0.0,2648.0,14063.0,2,12,0


In [24]:
daily_features['rolling_3day_avg'] = daily_features['daily_kwh'].rolling(window=3).mean()

daily_features['lag_1'] = daily_features['daily_kwh'].shift(1)

In [25]:
daily_features.head()

,datetime,daily_kwh,avg_voltage,avg_intensity,kitchen_kwh,laundry_kwh,other_kwh,day_of_week,month,is_weekend,rolling_3day_avg,lag_1
0,2006-12-16,20.152933,236.243763,13.082828,0.0,546.0,4926.0,5,12,1,NaN,NaN
1,2006-12-17,56.507667,240.087028,9.999028,2033.0,4187.0,13341.0,6,12,1,NaN,20.152933
2,2006-12-18,36.730433,241.231694,6.421667,1063.0,2621.0,14018.0,0,12,0,37.797011,56.507667
3,2006-12-19,27.769900,241.999313,4.926389,839.0,7602.0,6197.0,1,12,0,40.336000,36.730433
4,2006-12-20,37.095800,242.308062,6.467361,0.0,2648.0,14063.0,2,12,0,33.865378,27.769900


In [26]:
daily_features.dropna(inplace=True)
daily_features.isnull().sum()

datetime            0
daily_kwh           0
avg_voltage         0
avg_intensity       0
kitchen_kwh         0
laundry_kwh         0
other_kwh           0
day_of_week         0
month               0
is_weekend          0
rolling_3day_avg    0
lag_1               0
dtype: int64

In [29]:
import holidays

In [40]:
fr_holidays = holidays.France(years=[2006, 2007, 2008, 2009, 2010])

daily_features['is_holiday'] = daily_features['datetime'].isin(fr_holidays).astype(int)

C:\Users\chaud\AppData\Local\Temp\ipykernel_18372\1777568167.py:3: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  daily_features['is_holiday'] = daily_features['datetime'].isin(fr_holidays).astype(int)


In [41]:
daily_features['is_holiday'].unique()

array([0, 1])

In [45]:
daily_features.head()

,datetime,daily_kwh,avg_voltage,avg_intensity,kitchen_kwh,laundry_kwh,other_kwh,day_of_week,month,is_weekend,rolling_3day_avg,lag_1,is_holiday
2,2006-12-18,36.730433,241.231694,6.421667,1063.0,2621.0,14018.0,0,12,0,37.797011,56.507667,0
3,2006-12-19,27.769900,241.999313,4.926389,839.0,7602.0,6197.0,1,12,0,40.336000,36.730433,0
4,2006-12-20,37.095800,242.308062,6.467361,0.0,2648.0,14063.0,2,12,0,33.865378,27.769900,0
5,2006-12-21,28.610400,241.040549,5.033797,1765.0,2623.0,10421.0,3,12,0,31.158700,37.095800,0
6,2006-12-22,39.022300,241.184549,6.872917,3151.0,350.0,11131.0,4,12,0,34.909500,28.610400,0


In [43]:
daily_features.to_csv('final_energy_features.csv', index=False)